In [1]:
import pandas as pd
import numpy as np


In [2]:
contract_data = pd.read_csv("Resources/nba_salaries2019.csv")
print(contract_data)

      Rk             Player   Tm       2019-20       2020-21       2021-22  \
0      1      Stephen Curry  GSW  $40,231,758   $43,006,362   $45,780,966    
1      2         Chris Paul  OKC  $38,506,482   $41,358,814   $44,211,146    
2      3  Russell Westbrook  HOU  $38,178,000   $41,006,000   $43,848,000    
3      4          John Wall  WAS  $37,800,000   $40,824,000   $43,848,000    
4      5       James Harden  HOU  $37,800,000   $40,824,000   $43,848,000    
..   ...                ...  ...           ...           ...           ...   
563  564     William Howard  UTA      $50,000            NaN           NaN   
564  565      Isaiah Taylor  TOR      $50,000            NaN           NaN   
565  566        Tyler Lydon  SAC      $50,000            NaN           NaN   
566  567    Hollis Thompson  SAC      $50,000            NaN           NaN   
567  568         Tyler Ulis  SAC      $50,000            NaN           NaN   

          2022-23 2023-24 2024-25 Signed Using     Guaranteed  

In [3]:
player_data = pd.read_csv("Resources/player_stats.csv")

player_data

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
3,4,Kyle Alexander,PF,23,MIA,2,0,13,1,2,...,NaN,2,1,3,0,0,0,1,1,2
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,525,Trae Young,PG,21,ATL,60,60,2120,546,1249,...,0.860,32,223,255,560,65,8,289,104,1778
647,526,Cody Zeller,C,27,CHO,58,39,1341,251,479,...,0.682,160,251,411,88,40,25,75,140,642
648,527,Tyler Zeller,C,30,SAS,2,0,4,1,4,...,NaN,3,1,4,0,0,0,0,0,2
649,528,Ante Žižić,C,23,CLE,22,0,221,41,72,...,0.737,18,48,66,6,7,5,10,27,96


In [4]:
player_contracts = pd.merge(contract_data, player_data, on="Player", how = "inner")
print(player_contracts)

     Rk_x             Player Tm_x       2019-20       2020-21       2021-22  \
0       1      Stephen Curry  GSW  $40,231,758   $43,006,362   $45,780,966    
1       2         Chris Paul  OKC  $38,506,482   $41,358,814   $44,211,146    
2       3  Russell Westbrook  HOU  $38,178,000   $41,006,000   $43,848,000    
3       5       James Harden  HOU  $37,800,000   $40,824,000   $43,848,000    
4       6       LeBron James  LAL  $37,436,858   $39,219,565   $41,002,273    
..    ...                ...  ...           ...           ...           ...   
662   560     Marques Bolden  CLE      $50,752            NaN           NaN   
663   562        Paul Watson  ATL      $50,752            NaN           NaN   
664   562        Paul Watson  ATL      $50,752            NaN           NaN   
665   562        Paul Watson  ATL      $50,752            NaN           NaN   
666   564     William Howard  UTA      $50,000            NaN           NaN   

          2022-23 2023-24 2024-25 Signed Using  ...

In [5]:
player_contracts.drop(["2020-21", "2021-22", "2022-23", "2023-24", "2024-25"], axis=1)


,Rk_x,Player,Tm_x,2019-20,Signed Using,Guaranteed,Rk_y,Pos,Age,Tm_y,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Stephen Curry,GSW,"$40,231,758",Bird Rights,"$129,019,086",121,PG,31,GSW,...,1.000,4,22,26,33,5,2,16,11,104
1,2,Chris Paul,OKC,"$38,506,482",Bird Rights,"$79,865,296",396,PG,34,OKC,...,0.907,26,323,349,472,111,11,161,158,1232
2,3,Russell Westbrook,HOU,"$38,178,000",Bird Rights,"$123,032,000",506,PG,31,HOU,...,0.763,100,351,451,401,93,20,255,200,1553
3,5,James Harden,HOU,"$37,800,000",Bird Rights,"$122,472,000",198,SG,30,HOU,...,0.865,70,376,446,512,125,60,308,227,2335
4,6,LeBron James,LAL,"$37,436,858",Cap Space,"$76,656,423",250,PG,35,LAL,...,0.693,66,459,525,684,78,36,261,118,1698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,560,Marques Bolden,CLE,"$50,752",NaN,"$50,752",52,C,21,CLE,...,NaN,0,2,2,0,1,0,0,1,0
663,562,Paul Watson,ATL,"$50,752",NaN,"$50,752",504,SF-SG,25,TOT,...,0.778,2,15,17,8,4,1,3,5,31
664,562,Paul Watson,ATL,"$50,752",NaN,"$50,752",504,SG,25,ATL,...,NaN,0,2,2,3,1,0,0,1,0
665,562,Paul Watson,ATL,"$50,752",NaN,"$50,752",504,SF,25,TOR,...,0.778,2,13,15,5,3,1,3,4,31


In [6]:
# player_contracts["Cost per Point"] = player_contracts["PTS"]/player_contracts["2019-20"]
player_contracts

,Rk_x,Player,Tm_x,2019-20,2020-21,2021-22,2022-23,2023-24,2024-25,Signed Using,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Stephen Curry,GSW,"$40,231,758","$43,006,362","$45,780,966",NaN,NaN,NaN,Bird Rights,...,1.000,4,22,26,33,5,2,16,11,104
1,2,Chris Paul,OKC,"$38,506,482","$41,358,814","$44,211,146",NaN,NaN,NaN,Bird Rights,...,0.907,26,323,349,472,111,11,161,158,1232
2,3,Russell Westbrook,HOU,"$38,178,000","$41,006,000","$43,848,000","$46,662,000",NaN,NaN,Bird Rights,...,0.763,100,351,451,401,93,20,255,200,1553
3,5,James Harden,HOU,"$37,800,000","$40,824,000","$43,848,000","$46,872,000",NaN,NaN,Bird Rights,...,0.865,70,376,446,512,125,60,308,227,2335
4,6,LeBron James,LAL,"$37,436,858","$39,219,565","$41,002,273",NaN,NaN,NaN,Cap Space,...,0.693,66,459,525,684,78,36,261,118,1698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,560,Marques Bolden,CLE,"$50,752",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,2,2,0,1,0,0,1,0
663,562,Paul Watson,ATL,"$50,752",NaN,NaN,NaN,NaN,NaN,NaN,...,0.778,2,15,17,8,4,1,3,5,31
664,562,Paul Watson,ATL,"$50,752",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,2,2,3,1,0,0,1,0
665,562,Paul Watson,ATL,"$50,752",NaN,NaN,NaN,NaN,NaN,NaN,...,0.778,2,13,15,5,3,1,3,4,31


In [7]:
nba2019 = contract_data[["Player", 'Tm', "2019-20"]]
nba2019

# hina/ amber, just need to save this to CSV
nba2019.to_csv("Resources/nba2019_contract.csv")